In [266]:
import pandas as pd
import numpy as np

In [267]:
transactions = pd.read_csv("../data/transactions.csv", sep=";", decimal=",")
merchants = pd.read_csv("../data/merchants.csv", sep=";", decimal=",")

In [268]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   citizenid    99999 non-null  int64  
 1   cost         99999 non-null  float64
 2   countertype  99999 non-null  object 
 3   date         99999 non-null  object 
 4   fraudbits    99999 non-null  int64  
 5   logid        99999 non-null  int64  
 6   marketid     99999 non-null  int64  
 7   merchantid   99999 non-null  int64  
 8   model_cost   99999 non-null  float64
dtypes: float64(2), int64(5), object(2)
memory usage: 6.9+ MB


In [269]:
valid_transactions = transactions[transactions["fraudbits"] == 0]
valid_transactions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98719 entries, 0 to 99998
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   citizenid    98719 non-null  int64  
 1   cost         98719 non-null  float64
 2   countertype  98719 non-null  object 
 3   date         98719 non-null  object 
 4   fraudbits    98719 non-null  int64  
 5   logid        98719 non-null  int64  
 6   marketid     98719 non-null  int64  
 7   merchantid   98719 non-null  int64  
 8   model_cost   98719 non-null  float64
dtypes: float64(2), int64(5), object(2)
memory usage: 7.5+ MB


In [270]:
merchants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13158 entries, 0 to 13157
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   coef        13158 non-null  float64
 1   guild       13158 non-null  object 
 2   merchantid  13158 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 308.5+ KB


In [271]:
t_m = pd.merge(valid_transactions, merchants, on="merchantid", how="inner")
t_m.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98719 entries, 0 to 98718
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   citizenid    98719 non-null  int64  
 1   cost         98719 non-null  float64
 2   countertype  98719 non-null  object 
 3   date         98719 non-null  object 
 4   fraudbits    98719 non-null  int64  
 5   logid        98719 non-null  int64  
 6   marketid     98719 non-null  int64  
 7   merchantid   98719 non-null  int64  
 8   model_cost   98719 non-null  float64
 9   coef         98719 non-null  float64
 10  guild        98719 non-null  object 
dtypes: float64(3), int64(5), object(3)
memory usage: 8.3+ MB


In [272]:
t_m.head()

,citizenid,cost,countertype,date,fraudbits,logid,marketid,merchantid,model_cost,coef,guild
0,8315932911717591684,7.39,buy,924-09-27 12:57,0,5694353424282632895,203,100299578,30.0,0.46,Гильдия Парфюмеров
1,831575341702446221,22.47,sell,924-09-27 15:48,0,5697043883537282758,1682513,445135,80.0,0.57,Ассоциация Оливковых Торговцев
2,8316302311724840691,9.81,buy,924-09-27 18:51,0,5699920682275914112,1820486,57512,690.0,0.86,Гильдия Обработчиков Шерсти
3,8316277211719765682,6.21,buy,924-09-27 20:05,0,5701086315294310757,203,90356927,32.5,0.42,Гильдия Обработчиков Шерсти
4,5107559044524868241,41.04,buy,924-09-27 05:55,0,5687717990978109338,183,64197274,450.0,0.50,Ассоциация Оливковых Торговцев


In [273]:
sales = t_m[t_m['countertype'] == 'sell']
buys = t_m[t_m['countertype'] == 'buy']

sell_counts = sales.groupby('guild').size()
buy_counts = buys.groupby('guild').size()

ratio = sell_counts / buy_counts
ratio.sort_values(ascending=False)

guild
Союз Каменотесов                    0.125455
Лига Стеклодувов                    0.125000
Союз Судостроителей                 0.122625
Гильдия Пекарей                     0.113617
Братство Поставщиков Специй         0.113253
Лига Керамистов                     0.111820
Братство Виноделов                  0.111808
Объединение Фруктовых Садовников    0.110625
Ассоциация Оливковых Торговцев      0.107614
Гильдия Обработчиков Шерсти         0.107331
Братство Кузнецов                   0.106963
Гильдия Августовских Торговцев      0.105953
Объединение Рыбаков                 0.105563
Союз Скотоводов                     0.105263
Гильдия Парфюмеров                  0.104556
Гильдия Гончаров                    0.104395
Гильдия Мукомолов                   0.094317
Общество Ювелиров                   0.065693
dtype: float64

In [274]:
t_m['amn'] = np.abs(t_m['cost'] - t_m['model_cost']) / t_m['coef']
amn = t_m.groupby('guild').mean('amn')['amn']

eff = ratio * amn
eff.sort_values(ascending=False)

guild
Братство Кузнецов                   196.355958
Братство Виноделов                  193.031781
Объединение Фруктовых Садовников    154.031733
Гильдия Обработчиков Шерсти         145.231554
Союз Каменотесов                    126.410274
Гильдия Пекарей                     116.603193
Лига Керамистов                     105.405690
Гильдия Августовских Торговцев       99.692990
Гильдия Парфюмеров                   96.872213
Союз Судостроителей                  90.625868
Ассоциация Оливковых Торговцев       73.492571
Гильдия Гончаров                     70.714000
Братство Поставщиков Специй          62.856791
Объединение Рыбаков                  45.960131
Гильдия Мукомолов                    42.193482
Союз Скотоводов                      39.890346
Общество Ювелиров                    36.191350
Лига Стеклодувов                     20.935466
dtype: float64

In [275]:
guild = t_m[t_m['guild'] == 'Братство Кузнецов']
guild = guild[guild['countertype'] == 'sell']
guild['hour'] = guild['date'].str.split().str[1].str.split(':').str[0].astype(int)
guild.head()

,citizenid,cost,countertype,date,fraudbits,logid,marketid,merchantid,model_cost,coef,guild,amn,hour
95,1953395777835778130,37.12,sell,924-09-27 19:06,0,5700167659210753589,2427358,97943637,375.00,0.18,Братство Кузнецов,1877.111111,19
311,3358853221727247984,2.65,sell,924-09-27 00:30,0,5682615481518342785,203,10094301,36.00,0.09,Братство Кузнецов,370.555556,0
419,1896869501593448462,5.76,sell,924-09-27 13:31,0,5694886187368265839,8845410,97771991,150.00,0.25,Братство Кузнецов,576.960000,13
492,1192779761638385915,111.58,sell,924-09-27 08:21,0,5690021001541009765,183,303478799,400.00,0.05,Братство Кузнецов,5768.400000,8
609,1704222841720179491,68.09,sell,924-09-27 11:57,0,5693419315629392368,2,1987896,82.88,0.34,Братство Кузнецов,43.500000,11


In [276]:
hours = guild.groupby('hour').size()
hours.sort_values(ascending=False)

hour
21    48
13    44
20    43
17    39
8     38
10    38
19    36
14    35
7     34
16    33
15    32
12    31
22    30
9     30
11    30
18    29
23    24
6     17
0     15
5     12
3      6
2      6
4      5
1      4
dtype: int64